In [1]:
%matplotlib inline
import itertools
import os
os.environ['CUDA_VISIBLE_DEVICES']=""
import numpy as np
import gpflow
import gpflow.training.monitor as mon
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf

# Demo: `gpflow.training.monitor`
In this notebook we'll demo how to use `gpflow.training.monitor` for logging the optimisation of a GPflow model.

## Creating the GPflow model
We first generate some random data and create a GPflow model.

Under the hood, GPflow gives a unique name to each model which is used to name the Variables it creates in the TensorFlow graph containing a random identifier. This is useful in interactive sessions, where people may create a few models, to prevent variables with the same name conflicting. However, when loading the model, we need to make sure that the names of all the variables are exactly the same as in the checkpoint. This is why we pass name="SVGP" to the model constructor, and why we use gpflow.defer_build().

In [2]:
np.random.seed(0)
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(Xt) + np.random.randn(*Xt.shape)

with gpflow.defer_build():
    m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(),
                           Z=np.linspace(0, 10, 5)[:, None],
                           minibatch_size=100, name="SVGP")
    m.likelihood.variance = 0.01
m.compile()

Let's compute log likelihood before the optimisation

In [3]:
print('LML before the optimisation: %f' % m.compute_log_likelihood())

LML before the optimisation: -1271605.621944


We will be using a TensorFlow optimiser. All TensorFlow optimisers have a support for `global_step` variable. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

The code below creates this variable using a monitor's helper function. It is important to create it before building the monitor in case the monitor includes a checkpoint task. This is because the checkpoint internally uses the TensorFlow Saver which creates a list of variables to save. Therefore all variables expected to be saved by the checkpoint task should exist by the time the task is created.

In [4]:
session = m.enquire_session()
global_step = mon.create_global_step(session)

## Construct the monitor

Next we need to construct the monitor. `gpflow.training.monitor` provides classes that are building blocks for the monitor. Essengially, a monitor is a function that is provided as a callback to an optimiser. It consists of a number of tasks that may be executed at each step, subject to their running condition.

In this example, we want to:
- log certain scalar parameters in TensorBoard,
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches,
- store a backup of the optimisation process periodically,
- log performance for a test set periodically.

We will define these tasks as follows:

In [5]:
print_task = mon.PrintTimingsTask().with_name('print')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

sleep_task = mon.SleepTask(0.01).with_name('sleep').with_name('sleep')

saver_task = mon.CheckpointTask('./monitor-saves').with_name('saver')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

model_tboard_task = mon.ModelToTensorBoardTask('./model-tensorboard', m).with_name('model_tboard')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

lml_tboard_task = mon.LmlToTensorBoardTask('./model-tensorboard', m).with_name('lml_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

As the above code shows, each task can be assigned a name and running conditions. The name will be shown in the task timing summary.

There are two different types of running conditions: `with_condition` controls execution of the task at each iteration in the optimisation loop. `with_exit_condition` is a simple boolean flag indicating that the task should also run at the end of optimisation.
In this example we want to run our tasks periodically, at every iteration or every 10th or 100th iteration.


## Custom tasks
We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, we may want to compute the performance on a test set. Here we create such a class by extending `BaseTensorBoardTask` to log the testing benchmarks in addition to all the scalar parameters.

In [6]:
class CustomTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, event_path, model, Xt, Yt):
        super().__init__(event_path, model)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:
        minibatch_size = 100
        preds = np.vstack([self.model.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        self._eval_summary(context, {self._full_test_err: test_err, self._full_test_nlpp: 0.0})

        
custom_tboard_task = CustomTensorBoardTask('./model-tensorboard', m, Xt, Yt).with_name('custom_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

Now we can put all these tasks into a monitor.

In [7]:
monitor_tasks = [print_task, model_tboard_task, lml_tboard_task, custom_tboard_task, saver_task, sleep_task]
monitor = mon.Monitor(monitor_tasks, session, global_step)

## Running the optimisation
We finally get to running the optimisation.

We may want to continue a previously run optimisation by resotring the TensorFlow graph from the latest checkpoint. Otherwise skip this step.

In [8]:
if os.path.isdir('./monitor-saves'):
    mon.restore_session(session, './monitor-saves')

In [9]:
optimiser = gpflow.train.AdamOptimizer(0.01)
with mon.Monitor(monitor_tasks, session, global_step, print_summary=True) as monitor:
    optimiser.minimize(m, step_callback=monitor, maxiter=450, global_step=global_step)

Iteration 10	total itr.rate 12.97/s	recent itr.rate 12.97/s	opt.step 10	total opt.rate 14.70/s	recent opt.rate 14.70/s
Iteration 20	total itr.rate 19.96/s	recent itr.rate 43.28/s	opt.step 20	total opt.rate 28.69/s	recent opt.rate 594.41/s
Iteration 30	total itr.rate 25.32/s	recent itr.rate 54.72/s	opt.step 30	total opt.rate 42.00/s	recent opt.rate 582.33/s
Iteration 40	total itr.rate 28.84/s	recent itr.rate 49.43/s	opt.step 40	total opt.rate 54.74/s	recent opt.rate 607.33/s
Iteration 50	total itr.rate 31.31/s	recent itr.rate 47.66/s	opt.step 50	total opt.rate 66.95/s	recent opt.rate 621.07/s
Iteration 60	total itr.rate 33.20/s	recent itr.rate 47.63/s	opt.step 60	total opt.rate 78.52/s	recent opt.rate 579.45/s
Iteration 70	total itr.rate 33.94/s	recent itr.rate 39.11/s	opt.step 70	total opt.rate 89.54/s	recent opt.rate 567.60/s
Iteration 80	total itr.rate 35.60/s	recent itr.rate 54.23/s	opt.step 80	total opt.rate 99.78/s	recent opt.rate 499.17/s
Iteration 90	total itr.rate 36.68/s	recen

 30%|███       | 30/100 [00:00<00:00, 298.97it/s]

Iteration 100	total itr.rate 37.52/s	recent itr.rate 47.16/s	opt.step 100	total opt.rate 118.52/s	recent opt.rate 419.90/s


100%|██████████| 100/100 [00:00<00:00, 403.01it/s]


Iteration 110	total itr.rate 31.96/s	recent itr.rate 12.89/s	opt.step 110	total opt.rate 128.16/s	recent opt.rate 686.31/s
Iteration 120	total itr.rate 32.88/s	recent itr.rate 47.90/s	opt.step 120	total opt.rate 136.86/s	recent opt.rate 541.90/s
Iteration 130	total itr.rate 33.74/s	recent itr.rate 49.42/s	opt.step 130	total opt.rate 145.71/s	recent opt.rate 649.17/s
Iteration 140	total itr.rate 34.59/s	recent itr.rate 51.44/s	opt.step 140	total opt.rate 153.58/s	recent opt.rate 516.43/s
Iteration 150	total itr.rate 35.24/s	recent itr.rate 47.73/s	opt.step 150	total opt.rate 161.31/s	recent opt.rate 546.01/s
Iteration 160	total itr.rate 35.86/s	recent itr.rate 48.64/s	opt.step 160	total opt.rate 168.61/s	recent opt.rate 524.00/s
Iteration 170	total itr.rate 36.40/s	recent itr.rate 48.14/s	opt.step 170	total opt.rate 175.44/s	recent opt.rate 498.73/s
Iteration 180	total itr.rate 36.85/s	recent itr.rate 46.42/s	opt.step 180	total opt.rate 181.36/s	recent opt.rate 425.72/s
Iteration 190	to

...

Iteration 410	total itr.rate 36.67/s	recent itr.rate 15.96/s	opt.step 410	total opt.rate 280.64/s	recent opt.rate 603.22/s
Iteration 420	total itr.rate 36.87/s	recent itr.rate 47.21/s	opt.step 420	total opt.rate 283.57/s	recent opt.rate 495.71/s
Iteration 430	total itr.rate 37.06/s	recent itr.rate 47.26/s	opt.step 430	total opt.rate 286.14/s	recent opt.rate 462.23/s
Iteration 440	total itr.rate 37.28/s	recent itr.rate 50.21/s	opt.step 440	total opt.rate 288.87/s	recent opt.rate 490.15/s
Iteration 450	total itr.rate 37.46/s	recent itr.rate 47.63/s	opt.step 450	total opt.rate 291.69/s	recent opt.rate 510.71/s


 49%|████▉     | 49/100 [00:00<00:00, 482.68it/s]

Iteration 450	total itr.rate 36.68/s	recent itr.rate 0.00/s	opt.step 450	total opt.rate 264.76/s	recent opt.rate 0.00/s


100%|██████████| 100/100 [00:00<00:00, 472.03it/s]


Tasks execution time summary:
print:	0.0445 (sec)
model_tboard:	0.1616 (sec)
lml_tboard:	1.3489 (sec)
custom_tboard:	1.2753 (sec)
saver:	3.6596 (sec)
sleep:	4.5392 (sec)


Now lets compute the log likelihood again. Hopefully we will see an increase in its value

In [10]:
print('LML after the optimisation: %f' % m.compute_log_likelihood())

LML after the optimisation: -68705.124191
